In [58]:
def features_probability(pan_series,lenth):
    """
    Compute a specific feature probability given that panda series
    Input: pan_series  'A panda series contains all data for that feature'
    Output: feature_prob  'A dictionay with key as data value and a probaility corresponding to probability of occurance'
            that repesent 'P(xi|y)' in Naive Bayes
    Example:
    >>> features_probability(df['loan_type'])
    >>> '{1: 0.7726327301673637,
         2: 0.148994405648276,
         3: 0.07526288084019156,
         4: 0.0031099833441687666}'
    """
    assert isinstance(pan_series,pd.core.series.Series)
    return dict(pan_series.value_counts()/lenth)


def train_feature_probability(df):
    """
    Compute all features'probability as a list of dictionaries
    Input: 'df' training_data dataframe 
    Output: List of dictionaries that represens joint distribution of all features 'P(x|y)'
    """
    assert isinstance(df,pd.core.frame.DataFrame)
    train_features_list = [df['loan_type'], df['property_type'], df['loan_purpose'], 
                    df['owner_occupancy'],
                    df['preapproval'], 
                    df['applicant_ethnicity'], df['applicant_race_1'],df['applicant_sex'], df['loan_amount_000s']]#,df['action_taken']]
    df = pd.concat(train_features_list,axis = 1)
    df = df.dropna()
    return [features_probability(df.iloc[:,i],len(df)) for i in range(len(df.columns)-1)] ,df


def loan_prediction(x,df_a,df_na):
    """
    Given a loan applicant information as a pd.series. Compute how likely he/she will get the loan
    Input: 'df_a' A dataframe that contians all Approved datasets
            'df_na' A dataframe that contians all Unapproved datasets
    Output: A string that that represents the predicted loan application result
    """
    assert isintance(x,pd.core.series.Series)
    approved_dict, df_a= train_feature_probability(df_a)
    not_approved_dict,df_na = train_feature_probability(df_na)
    
    P_a = len(df_a)/(len(df_a)+len(df_na))
    P_na = len(df_na)/(len(df_a)+len(df_na))
    def get_prob_of_loan_from_approved(x):
        """
        Get the probability of loan amount x from approved datasets
        """
        d = np.array(df_a['loan_amount_000s'].astype(int))
        return ((d> math.floor(x)-5).sum() - (d >math.ceil(x)+5).sum() )/ len(d)
    
    def get_prob_of_loan_from_unapproved(x):
        """
        Get the probability of loan amount x from un_approved datasets
        """
        d = np.array(df_na['loan_amount_000s'].astype(int))
        return ((d> math.floor(x)-5).sum() - (d >math.ceil(x)+5).sum() )/ len(d)
    
    test_features_list = ['loan_type' ,'property_type','loan_purpose', 'owner_occupancy','preapproval', 
                    'applicant_ethnicity','applicant_race_1','applicant_sex', 'loan_amount_000s']
    
    array_of_x = [x[i] for i in test_features_list]
    p_x_a = 0
    p_x_na = 0
    for count,item in enumerate(array_of_x):
        if count != len(array_of_x)-2:
            p_x_a += approved_dict[count][item]
            p_x_na += not_approved_dict[count][item]
        else:
            p_x_a += get_prob_of_loan_from_approved(item)
            p_x_na += get_prob_of_loan_from_unapproved(item)
            break
    p_x_a += P_a
    p_x_na += P_na
    return  "Higher Probability You wil Get the Loan" if p_x_a> p_x_na else "Higher Probability You wil not Get the Loan"
    

In [63]:
import pandas as pd
import numpy as np
import math

df = pd.read_csv('hmda_2017_ca_all-records_labels/hmda_2017_ca_all-records_labels.csv')
a = df.groupby('action_taken')
t1 = a.get_group(1)
t2 = a.get_group(2)
t8 = a.get_group(8)
t3 = a.get_group(3)
t7 = a.get_group(7)

approved = pd.concat([t1, t2, t8], ignore_index=True)
not_approved = pd.concat([t3,t7], ignore_index=True)
approved.to_csv('approved.csv',index =False)
not_approved.to_csv('not_approved.csv',index =False)

C:\Users\liuzh\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (34,36,38,42,44,46,48) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [64]:

df_a = pd.read_csv('approved.csv')
df_na = pd.read_csv('not_approved.csv')

C:\Users\liuzh\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (34,36,38,44,46,48) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\liuzh\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (36,38,44,46,48) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [65]:
loan_prediction(df.iloc[1],df_a,df_na)

'Higher Probability You wil Get the Loan'

In [62]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline


d = np.array(df['loan_amount_000s'].astype(int))

plt.hist(d, bins=200,range=(1,4000))
plt.gca().set(title='Frequency Histogram', ylabel='Frequency');
#plt.gca().set_xlim(d.min(), d.max())
plt.show()
plt.savefig('loan_histogram')

ValueError: Cannot convert non-finite values (NA or inf) to integer